In [1]:
!pwd

/home/yangzho6/GRIFFIN2


In [2]:
import torch 
import transformers 
import lm_eval 
from datasets import load_dataset 
from transformers import AutoTokenizer, LlamaForCausalLM 
import numpy as np 

In [3]:
dataset = load_dataset("tasksource/bigbench", "date_understanding") 

/home/yangzho6/anaconda3/envs/griffin/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for tasksource/bigbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tasksource/bigbench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
!export CUDA_VISIBLE_DEVICES=8

In [3]:
!gpustat

lovelace.ece.local.cmu.edu  Sun Jun  9 07:07:17 2024  545.23.06
[0] NVIDIA L40 | 29°C,   0 % |     4 / 46068 MB |
[1] NVIDIA L40 | 32°C,   0 % |     4 / 46068 MB |
[2] NVIDIA L40 | 32°C,   0 % |     4 / 46068 MB |
[3] NVIDIA L40 | 31°C,   0 % |     4 / 46068 MB |
[4] NVIDIA L40 | 33°C,   0 % |     4 / 46068 MB |
[5] NVIDIA L40 | 28°C,   0 % |     4 / 46068 MB |
[6] NVIDIA L40 | 33°C,   0 % |     4 / 46068 MB |
[7] NVIDIA L40 | 32°C,   0 % |     4 / 46068 MB |
[8] NVIDIA L40 | 32°C,   0 % |     4 / 46068 MB |
[9] NVIDIA L40 | 33°C,   0 % |     4 / 46068 MB |


In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct") 
if tokenizer.pad_token is not None: 
    print("tokenizer has pad token {}".format(tokenizer.pad_token)) 
else: 
    tokenizer.pad_token = tokenizer.eos_token 
    print("We now use eos_token as pad token") 
tokenizer.padding_side = "left" 
model = LlamaForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map = "cuda", torch_dtype = torch.bfloat16) 

/home/yangzho6/anaconda3/envs/griffin/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now use eos_token as pad token


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
chainofthoughtprompt = "Q: In the US, Thanksgiving is on the fourth Thursday of November. Today is the US Thanksgiving of 2001. What is the date today in MM/DD/YYYY? \
A: The first day of November 2001 or 11/01/2001 is a Thursday, so the first Thursday is on 11/01/2001. The second is on 11/08/2001. The third is on 11/15/2001. So the fourth is on 11/22/2001. So the answer is 11/22/2001.\
\n\n\
Q: Jane scheduled 3 apointments with 5 poeple for tomorrow (Tue, 7/9/1972). What is the date one year ago from today in MM/DD/YYYY? \
A: If tomorrow is 7/9/1972, then today is 7/8/1972, and one year ago from today is 7/8/1971. So the answer is 7/8/1971.\
\n\n\
Q: Yesterday, Jan 21, 2011, Jane ate 2 pizzas and 5 wings. What is the date one week from today in MM/DD/YYYY? \
A: If yesterday is 01/21/2011, then today is 01/22/2011, and one week from today is 7 days in the future of today, which is 01/29/2011. So the answer is 01/29/2011.\
\n\n\
Q: Today is the second day of the third month of 1966. What is the date one week ago from today in MM/DD/YYYY? \
A: If today is in the third month of 1966, then today is in March 1966. Also, today is the second day of March, so today is 03/02/1966. One week ago from today is 7 days in the past of today, which is the 5th last day of February 1966. Since there are 28 days in February 1996, the 5th last day of February 1966 is 02/23/1966. So the answer is 02/23/1966.\
\n\n\
Q: Today, 8/3/1997, is a day that we will never forget. What is the date one week from today in MM/DD/YYYY? \
A: If today is 8/3/1997, then one week from today is 7 days in the future of today, which is 8/10/1997. So the answer is 8/10/1997.\
\n\n\
Q: This is the last day of 1899. What is the date one week ago from today in MM/DD/YYYY? \
A: If today is the last day of 1899, then today is 12/31/1899. One week ago from today is 7 days in the past of today, which is 12/24/1899. So the answer is 12/24/1899.\
\n\n\
Q: Today is the last day of the first quarter of 2008. What is the date one week from today in MM/DD/YYYY? \
A: One quarter is 3 months, so the first quarter is January, February, and March. Then, the last day of the first quarter of 2008 is the last day of March, which is 03/31/2008. One week from today is 7 days in the future of today, which is the 7th day of April 2008 or 04/07/2008. So the answer is 04/07/2008. \
\n\n\
Q: Today is Christmas Eve of 1937. What is the date yesterday in MM/DD/YYYY? \
A: If today is Chrstmas Eve of 1937, then today is 12/24/1937. Yesterday is 1 day in the past of today, which is 12/23/1937. So the answer is 12/23/1937.\
\n\n\
Q: Jane is celebrating the last day of Jan 2012. What is the date one week from today in MM/DD/YYYY? \
A: If today is the last day of Jan 2012, then today is 01/31/2012. One week from today is 7 day in the future of today, which is the 7th day of February 2012, or 02/07/2012. So the answer is 02/07/2012.\
\n\n\
Q: The day before yesterday was 11/23/1933. What is the date tomorrow in MM/DD/YYYY? \
A: If the day before yesterday was 11/23/1933, then yesterday is 11/24/1933. Today is 11/25/1933. Tomorrow is 1 day in the future of today, which is 11/26/1933. So the answer is 11/26/1933.\
\n\n" 

In [6]:
from termcolor import colored 
from datasets import concatenate_datasets 

In [29]:
dataset = concatenate_datasets([dataset["train"], dataset["validation"]]) 

In [48]:
for i in range(10, len(dataset)): 
    inputtext = chainofthoughtprompt + dataset[i]["inputs"] 
    inputids = tokenizer(inputtext, add_special_tokens = True, return_tensors = "pt") 
    inputids["input_ids"] = inputids["input_ids"].to(model.device) 
    output = model.generate(inputids["input_ids"], max_length = inputids["input_ids"].shape[-1] + 100, do_sample = False) 
    print(inputtext, end = "") 
    anw = tokenizer.decode(output[0][inputids["input_ids"].shape[-1] :], skip_special_tokens = False) 
    terminationidx = anw.find("Q: ") 
    # print(terminationidx) 
    print(colored(anw[: terminationidx], "green"), end = " ") 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: In the US, Thanksgiving is on the fourth Thursday of November. Today is the US Thanksgiving of 2001. What is the date today in MM/DD/YYYY? A: The first day of November 2001 or 11/01/2001 is a Thursday, so the first Thursday is on 11/01/2001. The second is on 11/08/2001. The third is on 11/15/2001. So the fourth is on 11/22/2001. So the answer is 11/22/2001.

Q: Jane scheduled 3 apointments with 5 poeple for tomorrow (Tue, 7/9/1972). What is the date one year ago from today in MM/DD/YYYY? A: If yesterday is 7/9/1972, then today is 7/8/1972, and one year ago from today is 7/8/1971. So the answer is 7/8/1971.

Q: Yesterday, Jan 21, 2011, Jane ate 2 pizzas and 5 wings. What is the date one week from today in MM/DD/YYYY? A: If yesterday is 01/21/2011, then today is 01/22/2011, and one week from today is 7 days in the future of today, which is 01/29/2011. So the answer is 01/29/2011.

Q: Today is the second day of the third month of 1966. What is the date one week ago from today in MM/DD/

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: In the US, Thanksgiving is on the fourth Thursday of November. Today is the US Thanksgiving of 2001. What is the date today in MM/DD/YYYY? A: The first day of November 2001 or 11/01/2001 is a Thursday, so the first Thursday is on 11/01/2001. The second is on 11/08/2001. The third is on 11/15/2001. So the fourth is on 11/22/2001. So the answer is 11/22/2001.

Q: Jane scheduled 3 apointments with 5 poeple for tomorrow (Tue, 7/9/1972). What is the date one year ago from today in MM/DD/YYYY? A: If yesterday is 7/9/1972, then today is 7/8/1972, and one year ago from today is 7/8/1971. So the answer is 7/8/1971.

Q: Yesterday, Jan 21, 2011, Jane ate 2 pizzas and 5 wings. What is the date one week from today in MM/DD/YYYY? A: If yesterday is 01/21/2011, then today is 01/22/2011, and one week from today is 7 days in the future of today, which is 01/29/2011. So the answer is 01/29/2011.

Q: Today is the second day of the third month of 1966. What is the date one week ago from today in MM/DD/

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: In the US, Thanksgiving is on the fourth Thursday of November. Today is the US Thanksgiving of 2001. What is the date today in MM/DD/YYYY? A: The first day of November 2001 or 11/01/2001 is a Thursday, so the first Thursday is on 11/01/2001. The second is on 11/08/2001. The third is on 11/15/2001. So the fourth is on 11/22/2001. So the answer is 11/22/2001.

Q: Jane scheduled 3 apointments with 5 poeple for tomorrow (Tue, 7/9/1972). What is the date one year ago from today in MM/DD/YYYY? A: If yesterday is 7/9/1972, then today is 7/8/1972, and one year ago from today is 7/8/1971. So the answer is 7/8/1971.

Q: Yesterday, Jan 21, 2011, Jane ate 2 pizzas and 5 wings. What is the date one week from today in MM/DD/YYYY? A: If yesterday is 01/21/2011, then today is 01/22/2011, and one week from today is 7 days in the future of today, which is 01/29/2011. So the answer is 01/29/2011.

Q: Today is the second day of the third month of 1966. What is the date one week ago from today in MM/DD/

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: In the US, Thanksgiving is on the fourth Thursday of November. Today is the US Thanksgiving of 2001. What is the date today in MM/DD/YYYY? A: The first day of November 2001 or 11/01/2001 is a Thursday, so the first Thursday is on 11/01/2001. The second is on 11/08/2001. The third is on 11/15/2001. So the fourth is on 11/22/2001. So the answer is 11/22/2001.

Q: Jane scheduled 3 apointments with 5 poeple for tomorrow (Tue, 7/9/1972). What is the date one year ago from today in MM/DD/YYYY? A: If yesterday is 7/9/1972, then today is 7/8/1972, and one year ago from today is 7/8/1971. So the answer is 7/8/1971.

Q: Yesterday, Jan 21, 2011, Jane ate 2 pizzas and 5 wings. What is the date one week from today in MM/DD/YYYY? A: If yesterday is 01/21/2011, then today is 01/22/2011, and one week from today is 7 days in the future of today, which is 01/29/2011. So the answer is 01/29/2011.

Q: Today is the second day of the third month of 1966. What is the date one week ago from today in MM/DD/

KeyboardInterrupt: 

In [49]:
dataset = load_dataset("tasksource/bigbench", "sports_understanding") 

/home/yangzho6/anaconda3/envs/griffin/lib/python3.12/site-packages/datasets/load.py:1486: FutureWarning: The repository for tasksource/bigbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tasksource/bigbench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [50]:
dataset = concatenate_datasets([dataset["train"], dataset["validation"]]) 

In [51]:
chainofthoughtprompt = "Q: Determine whether the following statement or statements are plausible or implausible: Statement: Marouane Fellaini scored a freekick Plausible/implausible? \
A: Marouane Fellaini is a Belgian professional soccer player, and freekicks are a common way to score goals in soccer. So the answer is plausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Javi Martinez comitted a three second violation Plausible/implausible? \
A: Javi Martinez is a Spanish professional soccer player, and three second violations are a common rule in basketball. So the answer is implausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Gerard Pique watched the pitch go by in the National League Championship Series Plausible/implausible? \
A: Gerard Pique is a Spanish professional soccer player, and the National League Championship Series is a baseball event. So the answer is implausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Julio Jones was called for the goal tend Plausible/implausible? \
A: Julio Jones is an American professional football player, and goal tend is a common rule in basketball. So the answer is implausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Lionel Messi scored a touchdown Plausible/implausible? \
A: Lionel Messi is an Argentine professional soccer player, and touchdowns are a common way to score points in American football. So the answer is implausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Cooper Kupp scored in the third period in the Stanley Cup Plausible/implausible? \
A: Cooper Kupp is an American professional football player, and the Stanley Cup is a ice hockey event. So the answer is implausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Pierre-Luc Dubois passed the puck Plausible/implausible? \
A: Pierre-Luc Dubois is a Canadian professional ice hockey player, and passing the puck is a common action in ice hockey. So the answer is plausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Yordan Alvarez grounded out to second base Plausible/implausible? \
A: Yordan Alvarez is a Cuban professional baseball player, and grounding out to second base is a common way to make an out in baseball. So the answer is plausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: George Springer committed a blocking foul Plausible/implausible? \
A: George Springer is an American professional baseball player, and blocking fouls are a common rule in basketball. So the answer is implausible. \
\n\n\
Q: Determine whether the following statement or statements are plausible or implausible: Statement: Baker Mayfield hit nothing but net Plausible/implausible? \
A: Baker Mayfield is an American professional football player, and hitting nothing but net is a common way to score in basketball. So the answer is implausible. \
\n\n" 


In [53]:
for i in range(10, len(dataset)): 
    inputtext = chainofthoughtprompt + "Q: " + dataset[i]["inputs"] + "A: " 
    inputids = tokenizer(inputtext, add_special_tokens = True, return_tensors = "pt") 
    inputids["input_ids"] = inputids["input_ids"].to(model.device) 
    output = model.generate(inputids["input_ids"], max_length = inputids["input_ids"].shape[-1] + 100, do_sample = False) 
    print(inputtext, end = "") 
    anw = tokenizer.decode(output[0][inputids["input_ids"].shape[-1] :], skip_special_tokens = False) 
    terminationidx = anw.find("Q: ") 
    # print(terminationidx) 
    print(colored(anw[: terminationidx], "green"), end = " ") 

/home/yangzho6/anaconda3/envs/griffin/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/yangzho6/anaconda3/envs/griffin/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe un

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Marouane Fellaini scored a freekick Plausible/implausible? A: Marouane Fellaini is a Belgian professional soccer player, and freekicks are a common way to score goals in soccer. So the answer is plausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Javi Martinez comitted a three second violation Plausible/implausible? A: Javi Martinez is a Spanish professional soccer player, and three second violations are a common rule in basketball. So the answer is implausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Gerard Pique watched the pitch go by in the National League Championship Series Plausible/implausible? A: Gerard Pique is a Spanish professional soccer player, and the National League Championship Series is a baseball event. So the answer is implausible. 

Q: Determine 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Determine whether the following statement or statements are plausible or implausible: Statement: Marouane Fellaini scored a freekick Plausible/implausible? A: Marouane Fellaini is a Belgian professional soccer player, and freekicks are a common way to score goals in soccer. So the answer is plausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Javi Martinez comitted a three second violation Plausible/implausible? A: Javi Martinez is a Spanish professional soccer player, and three second violations are a common rule in basketball. So the answer is implausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Gerard Pique watched the pitch go by in the National League Championship Series Plausible/implausible? A: Gerard Pique is a Spanish professional soccer player, and the National League Championship Series is a baseball event. So the answer is implausible. 

Q: Determine 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Determine whether the following statement or statements are plausible or implausible: Statement: Marouane Fellaini scored a freekick Plausible/implausible? A: Marouane Fellaini is a Belgian professional soccer player, and freekicks are a common way to score goals in soccer. So the answer is plausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Javi Martinez comitted a three second violation Plausible/implausible? A: Javi Martinez is a Spanish professional soccer player, and three second violations are a common rule in basketball. So the answer is implausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Gerard Pique watched the pitch go by in the National League Championship Series Plausible/implausible? A: Gerard Pique is a Spanish professional soccer player, and the National League Championship Series is a baseball event. So the answer is implausible. 

Q: Determine 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Determine whether the following statement or statements are plausible or implausible: Statement: Marouane Fellaini scored a freekick Plausible/implausible? A: Marouane Fellaini is a Belgian professional soccer player, and freekicks are a common way to score goals in soccer. So the answer is plausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Javi Martinez comitted a three second violation Plausible/implausible? A: Javi Martinez is a Spanish professional soccer player, and three second violations are a common rule in basketball. So the answer is implausible. 

Q: Determine whether the following statement or statements are plausible or implausible: Statement: Gerard Pique watched the pitch go by in the National League Championship Series Plausible/implausible? A: Gerard Pique is a Spanish professional soccer player, and the National League Championship Series is a baseball event. So the answer is implausible. 

Q: Determine 

KeyboardInterrupt: 

In [36]:
dataset = load_dataset("tau/commonsense_qa", split = "train") 

# randomly sample 10 examples 
np.random.seed(1) 
indices = np.random.choice(len(dataset), 10, replace = False) 

In [37]:
indices 

array([6514,  619, 3818, 3530, 1225, 5899, 1406, 5042, 2045, 4549])

In [38]:
datasetsmall = dataset.select(indices) 
for i, batch in enumerate(datasetsmall): 
    # options = batch["choices"] 
    options = batch["choices"]["text"] 
    print("Q: {}\nOptions: (a) {} (b) {} (c) {} (d) {} (e) {}".format(batch["question"], options[0], options[1], options[2], options[3], options[4])) 
    print("Correct Answer: {}\n".format(batch["answerKey"] + " " + options[batch["choices"]["label"].index(batch["answerKey"])])) 

Q: John heard a language that he could not understand. He thought that the door was shut, but he eventually realized that there was no door, and that the light source that was blinding his eyes was very familiar.  He was on his back looking at what?
Options: (a) sky (b) lamp (c) hallway (d) dard (e) closed room
Correct Answer: A sky

Q: The sun was out and the temperature changing rapidly, it was really beginning to what?
Options: (a) drop to zero (b) increase (c) cool down (d) dropping quickly (e) heat up
Correct Answer: E heat up

Q: When do girls eat ice cream on the couch?
Options: (a) wash hands (b) were hungry (c) depressed (d) athlete's foot (e) cool down
Correct Answer: C depressed

Q: Dan is driving a car, though he's stopped at a light right now. What else can we assume that he's doing?
Options: (a) listening to the radio (b) car crash (c) moving (d) get sleepy (e) going someplace
Correct Answer: E going someplace

Q: Where could you find a monkey in the wild?
Options: (a) fr

In [7]:
chainofthoughtprompt = "Q: John heard a language that he could not understand. He thought that the door was shut, but he eventually realized that there was no door, and that the light source that was blinding his eyes was very familiar.  He was on his back looking at what?\n\
Options: (a) sky (b) lamp (c) hallway (d) dard (e) closed room\n\n\
\
A: The answer must be a place where there are people speaking, no door, and also there is a light source that is intensively strong. lamp is a light source, but not intensively strong, hallway is a place with doors, and closed room also has some doors. Sky is the only place that has no door and a very strong light source, sun, during the day. Also, John can be on his back looking at the sky and hearing people speaking. So the answer is sky.\n\n\
\
Q: Q: The sun was out and the temperature changing rapidly, it was really beginning to what?\n\
Options: (a) drop to zero (b) increase (c) cool down (d) dropping quickly (e) heat up\n\n\
\
A: The answer must be a situation when the sun is out and temperature starts to shift because of the sun rise. The sun rise makes the temperature to go up, and temperature does not drop to zero, does not cool down, and does not drop quickly. Increase is ok but not as clear as heat up. So the answer is heat up.\n\n\
\
Q: When do girls eat ice cream on the couch?\n\
Options: (a) wash hands (b) were hungry (c) depressed (d) athlete's foot (e) cool down\n\n\
\
A: The answer must be a situation when eating ice cream and sitting on the couch help girls feel better. Eating ice cream and sitting on the couch are usually activies that make girls feel comfortable and pleasing. Washing hand is not related to eating ice cream. Being hungry is relevant, but ice cream is usually not the first thing to eat when hungry. Athlete's foot is not relevant to eating ice cream. When girls are cooled down, they won't do things that make them feel better. Feeling depressed is the most relevant among all and a strong reason for them to find ways to feel better by sitting on the couch and eating ice cream. So the answer is depressed.\n\n\
\
Q: Dan is driving a car, though he's stopped at a light right now. What else can we assume that he's doing?\n\
Options: (a) listening to the radio (b) car crash (c) moving (d) get sleepy (e) going someplace\n\n\
\
A: The answer must be a situation that doesn't infer more than the question has given. Dan is driving a car and stopped at a light, which is a common situation when driving. Listening to the radio is a common activity when driving but cannot be infered from Dan's actions. Car crash cannot be infered from Dan's actions. Dan stopped at a light, so he is not moving. Getting sleepy cannot be infered from the question. Going someplace is always right when driving, which is what Dan is doing. So the answer is going someplace.\n\n\
\
Q: Where could you find a monkey in the wild?\n\
Options: (a) friend's house (b) research laboratory (c) french government (d) beach (e) madagascar\n\n\
\
A: The answer must be a place of lots of monkeys in the wild to be found. A friend's house might have pets but not wild monkeys. A research laboratory might have monkeys but not in the wild. French government is irrelevant to finding wild monkeys, and wild monkeys usually cannot be found near the beaches. Madgascar is a place known for wild lives, including monkeys. So the answer is madagascar.\n\n\
\
Q: who uses a contraceptive device?\n\
Options: (a) pharmacy (b) bedroom (c) person (d) men (e) drug store\n\n\
\
A: The answer must be refering by whom the contraceptive device is used. A pharmacy is a place where contraceptive devices are sold, but cannot be refered by who. Bedroom is also a place and cannot be refered by who, and drug store is again a place and cannot be refered to by who. Men aren't the only ones who use contraceptive devices, and person is the most general term that can be refered to by who use contraceptive devices. So the answer is person.\n\n\
\
Q: Samantha decided to join the game, but there wasn't space. She had to stay on the waitlist until someone else did what?\n\
Options: (a) opt out (b) separate (c) opt out (d) leave (e) split apart\n\n\
\
A: The answer must be a situation when the number of players currently playing the game reaches the limit of the game, and someone has to exit the game for people from the waitlist to enter. Opt out is correct but repeated. Separate and split are not relevant to the players playing the game. Leave is the most relevant in the situation. So the answer is leave.\n\n\
\
Q: The storm was surprising, where didn't it show up for meteorologists?\n\
Options: (a) train (b) tropical seas (c) weather report (d) america (e) caribbean\n\n\
\
A: The answer must be a place where because it didn't show up, meteorologists were surprised. A train is irrelevant to the storm and meteorologists. Tropical seas are places where storms are common, so meteorologists wouldn't be surprised. America and caribbean are general places where storms can show up or not, and meteorologists wouldn't be surprised. If the storm didn't show up in the weather report, meteorologists would be surprised when the storm came. So the answer is weather report.\n\n\
\
Q: what color are rose flowers typically?\n\
Options: (a) vase (b) pink (c) red (d) blue (e) fragrant\n\n\
\
A: The answer must be a color that rose flowers typically have. A vase is not a color. Pink isn't the most typical color roses have. Blue is not a typical color roses have. Fragrant is not for describing the color of rose. Red is the most typical color that rose flowers have. So the answer is red.\n\n\
\
Q: You will find many swamps in the in the southern U.S. along what geographic feature?\n\
Options: (a) river delta (b) michigan (c) louisiana (d) wetlands (e) alligators\n\n\
\
A: The answer must to a geographic feature that is common in the southern U.S. and frequently appears with swamps. Michigan and Louisiana are the states in the U.S. not geographic features. Wetlands refer to the ecosystem of swamps, but not a geographic feature. Alligators live in swamps but not geographic features. River delta is a geographic feature that commonly appears with swamps. So the answer is river delta.\n\n" 

In [8]:
testdataset = load_dataset("tau/commonsense_qa", split = "test") 
for i, batch in enumerate(testdataset): 
    # inputtext = chainofthoughtprompt + "Q: " + dataset[i]["inputs"] + "A: " 
    options = batch["choices"]["text"] 
    inputtext = chainofthoughtprompt + "Q: {}\nOptions: (a) {} (b) {} (c) {} (d) {} (e) {}".format(batch["question"], options[0], options[1], options[2], options[3], options[4]) 
    inputids = tokenizer(inputtext, add_special_tokens = True, return_tensors = "pt") 
    inputids["input_ids"] = inputids["input_ids"].to(model.device) 
    # output = model.generate(inputids["input_ids"], max_length = inputids["input_ids"].shape[-1] + 100, do_sample = False) 
    output = model.generate(inputids["input_ids"], max_length = inputids["input_ids"].shape[-1] + 200, do_sample = False) 
    print(inputtext, end = "") 
    anw = tokenizer.decode(output[0][inputids["input_ids"].shape[-1] :], skip_special_tokens = False) 
    terminationidx = anw.find("Q: ") 
    # print(terminationidx) 
    print(colored(anw[: terminationidx], "green"), end = " ") 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: John heard a language that he could not understand. He thought that the door was shut, but he eventually realized that there was no door, and that the light source that was blinding his eyes was very familiar.  He was on his back looking at what?
Options: (a) sky (b) lamp (c) hallway (d) dard (e) closed room

A: The answer must be a place where there are people speaking, no door, and also there is a light source that is intensively strong. lamp is a light source, but not intensively strong, hallway is a place with doors, and closed room also has some doors. Sky is the only place that has no door and a very strong light source, sun, during the day. Also, John can be on his back looking at the sky and hearing people speaking. So the answer is sky.

Q: Q: The sun was out and the temperature changing rapidly, it was really beginning to what?
Options: (a) drop to zero (b) increase (c) cool down (d) dropping quickly (e) heat up

A: The answer must be a situation when the sun is out and te

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: John heard a language that he could not understand. He thought that the door was shut, but he eventually realized that there was no door, and that the light source that was blinding his eyes was very familiar.  He was on his back looking at what?
Options: (a) sky (b) lamp (c) hallway (d) dard (e) closed room

A: The answer must be a place where there are people speaking, no door, and also there is a light source that is intensively strong. lamp is a light source, but not intensively strong, hallway is a place with doors, and closed room also has some doors. Sky is the only place that has no door and a very strong light source, sun, during the day. Also, John can be on his back looking at the sky and hearing people speaking. So the answer is sky.

Q: Q: The sun was out and the temperature changing rapidly, it was really beginning to what?
Options: (a) drop to zero (b) increase (c) cool down (d) dropping quickly (e) heat up

A: The answer must be a situation when the sun is out and te

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: John heard a language that he could not understand. He thought that the door was shut, but he eventually realized that there was no door, and that the light source that was blinding his eyes was very familiar.  He was on his back looking at what?
Options: (a) sky (b) lamp (c) hallway (d) dard (e) closed room

A: The answer must be a place where there are people speaking, no door, and also there is a light source that is intensively strong. lamp is a light source, but not intensively strong, hallway is a place with doors, and closed room also has some doors. Sky is the only place that has no door and a very strong light source, sun, during the day. Also, John can be on his back looking at the sky and hearing people speaking. So the answer is sky.

Q: Q: The sun was out and the temperature changing rapidly, it was really beginning to what?
Options: (a) drop to zero (b) increase (c) cool down (d) dropping quickly (e) heat up

A: The answer must be a situation when the sun is out and te

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: John heard a language that he could not understand. He thought that the door was shut, but he eventually realized that there was no door, and that the light source that was blinding his eyes was very familiar.  He was on his back looking at what?
Options: (a) sky (b) lamp (c) hallway (d) dard (e) closed room

A: The answer must be a place where there are people speaking, no door, and also there is a light source that is intensively strong. lamp is a light source, but not intensively strong, hallway is a place with doors, and closed room also has some doors. Sky is the only place that has no door and a very strong light source, sun, during the day. Also, John can be on his back looking at the sky and hearing people speaking. So the answer is sky.

Q: Q: The sun was out and the temperature changing rapidly, it was really beginning to what?
Options: (a) drop to zero (b) increase (c) cool down (d) dropping quickly (e) heat up

A: The answer must be a situation when the sun is out and te

KeyboardInterrupt: 

In [13]:
dataset = load_dataset("tasksource/bigbench", "strategyqa", split = "train") 

# datasetnew = concatenate_datasets([dataset["train"], dataset["validation"]]) 

In [16]:
# randomly sample 10 examples 
np.random.seed(1) 
indices = np.random.choice(len(dataset), 10, replace = False) 

datasetsmall = dataset.select(indices) 
for i, batch in enumerate(datasetsmall): 
    # options = batch["choices"] 
    # print("Q: {}\nOptions: (a) {} (b) {} (c) {} (d) {} (e) {}".format(batch["question"], options[0], options[1], options[2], options[3], options[4])) 
    # print("Q: {}\nOptions: (a) {} (b) {} (c) {} (d) {} (e) {}".format(batch["question"], batch["choices"]["text"][0], batch["choices"]["text"][1], batch["choices"]["text"][2], batch["choices"]["text"][3], batch["choices"]["text"][4])) 
    print("Q: Yes or No: {}".format(batch["inputs"])) 
    # print("Correct Answer: {}\n".format(batch["answerKey"] + " " + options[batch["choices"]["label"].index(batch["answerKey"])])) 
    print("Correct Answer: {}\n".format(batch["targets"])) 

Q: Yes or No: Q: Can lobster breathe in the desert?
A:
Correct Answer: ['No. Lobsters use gills to breathe. Gills require water to breathe. There is no water in the desert.']

Q: Yes or No: Q: Could Edward Snowden join MENSA?
A:
Correct Answer: ['Yes. Snowden scored above 145 on two separate IQ tests. The minimum accepted IQ score for MENSA on the Stanford–Binet is 132, while for the Cattell it is 148.']

Q: Yes or No: Q: Would Harvey Milk have approved of Obama?
A:
Correct Answer: ['Yes. Obama awarded Harvey Milk a posthumous Medal of Freedom.  Obama was known for supporting marriage equality and LGBT rights.']

Q: Yes or No: Q: Could a snowy owl survive in the Sonoran?
A:
Correct Answer: ['No. The Snowy owl is a bird native to the Arctic regions of North America. Temperatures in the North American Arctic range from -36.4F to 50F. The Sonoran is one of the hottest deserts in the world. The Sonoran Desert can get as hot as 118F.']

Q: Yes or No: Q: Are the Great Lakes part of an intern

In [19]:
chainofthoughtprompt = "Q: Yes or No: Q: Can lobster breathe in the desert?\n\
A: Lobsters use gills to breathe. Gills require water to breathe. There is no water in the desert. So the answer is no.\n\n\
Q: Yes or No: Q: Could Edward Snowden join MENSA\n\
A: Snowden scored above 145 on two separate IQ tests. The minimum accepted IQ score for MENSA on the Stanford-Binet is 132, while for the Cattell it is 148. He passes the minimum requirement for MENSA. So the answer is yes.\n\n\
Q: Yes or No: Q: Would Harvey Milk have approved of Obama?\n\
A: Obama awarded Harvey Milk a posthumous Medal of Freedom.  Obama was known for supporting marriage equality and LGBT rights. Harvey Milk would have approved of Obama. So the answer is yes.\n\n\
Q: Yes or No: Q: Could a snowy owl survive in the Sonoran?\n\
A: The Snowy owl is a bird native to the Arctic regions of North America. Temperatures in the North American Arctic range from -36.4F to 50F. The Sonoran is one of the hottest deserts in the world. The Sonoran Desert can get as hot as 118F. So the answer is no.\n\n\
Q: Yes or No: Q: Are the Great Lakes part of an international border?\n\
A: The lakes are bordered on the north by Canada. The lakes are bordered on the south by United States. Canada and United States are two different countries. So the answer is yes.\n\n\
Q: Yes or No: Q: Can Justin Timberlake ride Shipwreck Falls at Six Flags?\n\
A: Shipwreck Falls is a boat ride at Six Flags. The minimum height for Shipwreck Falls is 42 inches. Justin Timberlake is 73 inches tall. So the answer is yes.\n\n\
Q: Yes or No: Q: Was animal in You're a Good Sport, Charlie Brown, hypothetically a hound?\n\
A: A hound is a type of hunting dog used to track prey. Hounds include Basenjis, Dachsunds, and Beagles, among others. Snoopy is the dog in the Charlie Brown movies and books. Snoopy is a Beagle. Beagles are a type of hound. So the answer is yes.\n\n\
Q: Yes or No: Q: Would an American feel lost due to language barriers at Disneyland Paris?\n\
A: All Disneyland Paris cast members are required to know and speak English. Travelers from England go to Disneyland Paris often without issue. Americans speak English and can communicate with the cast members at Disneyland Paris. So the answer is no.\n\n\
Q: Yes or No: Q: Are selfies more dangerous than plague in modern times?\n\
A: There are an average of 7 human plague cases reported each year according to the CDC. Selfies have caused people to fall off of cliffs while trying to get the perfect picture. From October 2011 and November 2017, there were 259 selfie deaths in 137 incidents. So the answer is yes.\n\n\
Q: Yes or No: Q: Would a Bulgarian priest eat a four-course meal on Christmas Eve?\n\
A: A four-course meal consists of a soup, an appetizer, an entrée, and dessert. The Bulgarian Christmas Eve meal has an odd number of dishes and an odd number of people sitting around the table. Four is an even number, not odd. So the answer is no.\n\n" 

In [20]:
# testdataset = load_dataset("tau/commonsense_qa", split = "test") 
testdataset = load_dataset("tasksource/bigbench", "strategyqa", split = "validation") 
for i, batch in enumerate(testdataset): 
    inputtext = chainofthoughtprompt + "Q: Yes or No: {}".format(batch["inputs"]) 
    inputids = tokenizer(inputtext, add_special_tokens = True, return_tensors = "pt") 
    inputids["input_ids"] = inputids["input_ids"].to(model.device) 
    # output = model.generate(inputids["input_ids"], max_length = inputids["input_ids"].shape[-1] + 100, do_sample = False) 
    output = model.generate(inputids["input_ids"], max_length = inputids["input_ids"].shape[-1] + 200, do_sample = False) 
    print(inputtext, end = "") 
    anw = tokenizer.decode(output[0][inputids["input_ids"].shape[-1] :], skip_special_tokens = False) 
    terminationidx = anw.find("Q: ") 
    # print(terminationidx) 
    print(colored(anw[: terminationidx], "green"), end = " ") 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Yes or No: Q: Can lobster breathe in the desert?
A: Lobsters use gills to breathe. Gills require water to breathe. There is no water in the desert. So the answer is no.

Q: Yes or No: Q: Could Edward Snowden join MENSA
A: Snowden scored above 145 on two separate IQ tests. The minimum accepted IQ score for MENSA on the Stanford-Binet is 132, while for the Cattell it is 148. He passes the minimum requirement for MENSA. So the answer is yes.

Q: Yes or No: Q: Would Harvey Milk have approved of Obama?
A: Obama awarded Harvey Milk a posthumous Medal of Freedom.  Obama was known for supporting marriage equality and LGBT rights. Harvey Milk would have approved of Obama. So the answer is yes.

Q: Yes or No: Q: Could a snowy owl survive in the Sonoran?
A: The Snowy owl is a bird native to the Arctic regions of North America. Temperatures in the North American Arctic range from -36.4F to 50F. The Sonoran is one of the hottest deserts in the world. The Sonoran Desert can get as hot as 118F. So

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Yes or No: Q: Can lobster breathe in the desert?
A: Lobsters use gills to breathe. Gills require water to breathe. There is no water in the desert. So the answer is no.

Q: Yes or No: Q: Could Edward Snowden join MENSA
A: Snowden scored above 145 on two separate IQ tests. The minimum accepted IQ score for MENSA on the Stanford-Binet is 132, while for the Cattell it is 148. He passes the minimum requirement for MENSA. So the answer is yes.

Q: Yes or No: Q: Would Harvey Milk have approved of Obama?
A: Obama awarded Harvey Milk a posthumous Medal of Freedom.  Obama was known for supporting marriage equality and LGBT rights. Harvey Milk would have approved of Obama. So the answer is yes.

Q: Yes or No: Q: Could a snowy owl survive in the Sonoran?
A: The Snowy owl is a bird native to the Arctic regions of North America. Temperatures in the North American Arctic range from -36.4F to 50F. The Sonoran is one of the hottest deserts in the world. The Sonoran Desert can get as hot as 118F. So

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Yes or No: Q: Can lobster breathe in the desert?
A: Lobsters use gills to breathe. Gills require water to breathe. There is no water in the desert. So the answer is no.

Q: Yes or No: Q: Could Edward Snowden join MENSA
A: Snowden scored above 145 on two separate IQ tests. The minimum accepted IQ score for MENSA on the Stanford-Binet is 132, while for the Cattell it is 148. He passes the minimum requirement for MENSA. So the answer is yes.

Q: Yes or No: Q: Would Harvey Milk have approved of Obama?
A: Obama awarded Harvey Milk a posthumous Medal of Freedom.  Obama was known for supporting marriage equality and LGBT rights. Harvey Milk would have approved of Obama. So the answer is yes.

Q: Yes or No: Q: Could a snowy owl survive in the Sonoran?
A: The Snowy owl is a bird native to the Arctic regions of North America. Temperatures in the North American Arctic range from -36.4F to 50F. The Sonoran is one of the hottest deserts in the world. The Sonoran Desert can get as hot as 118F. So

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Q: Yes or No: Q: Can lobster breathe in the desert?
A: Lobsters use gills to breathe. Gills require water to breathe. There is no water in the desert. So the answer is no.

Q: Yes or No: Q: Could Edward Snowden join MENSA
A: Snowden scored above 145 on two separate IQ tests. The minimum accepted IQ score for MENSA on the Stanford-Binet is 132, while for the Cattell it is 148. He passes the minimum requirement for MENSA. So the answer is yes.

Q: Yes or No: Q: Would Harvey Milk have approved of Obama?
A: Obama awarded Harvey Milk a posthumous Medal of Freedom.  Obama was known for supporting marriage equality and LGBT rights. Harvey Milk would have approved of Obama. So the answer is yes.

Q: Yes or No: Q: Could a snowy owl survive in the Sonoran?
A: The Snowy owl is a bird native to the Arctic regions of North America. Temperatures in the North American Arctic range from -36.4F to 50F. The Sonoran is one of the hottest deserts in the world. The Sonoran Desert can get as hot as 118F. So

KeyboardInterrupt: 